In [1]:
import os

import crystal_toolkit
from pymatgen.core import Structure
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.transformations.advanced_transformations import DopingTransformation

import matcalc
from matcalc import PhononCalc

In [ ]:
structure = Structure.from_file("Zn(FeO2)2_fd3m.cif")
print(structure)
structure

In [ ]:
sga_default = SpacegroupAnalyzer(structure)
sg_sym, sg_num = sga_default.get_space_group_symbol(), sga_default.get_space_group_number()

print(
    sg_sym, sg_num
)

In [ ]:
struc_super = structure.copy().make_supercell([2,2,2]) 
struc_super

In [ ]:
sga_default = SpacegroupAnalyzer(struc_super)
sg_sym, sg_num = sga_default.get_space_group_symbol(), sga_default.get_space_group_number()

print(
    sg_sym, sg_num
)

In [ ]:
dt = DopingTransformation("Mn2+", min_length=10)
lst_doping = dt.apply_transformation(struc_super, return_ranked_list=3)

for idx, ostruct in enumerate(lst_doping):
    print(f"-----Ordering {idx + 1}-----")
    print(ostruct)
    print("\n")

doped_structure = lst_doping[0]["structure"]
doped_structure

In [ ]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

sga_default = SpacegroupAnalyzer(doped_structure)
sg_sym, sg_num = sga_default.get_space_group_symbol(), sga_default.get_space_group_number()

print(
    sg_sym, sg_num
)

In [ ]:
sga_loose = SpacegroupAnalyzer(doped_structure, symprec=0.2, angle_tolerance=10)
sg_sym, sg_num = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()
print(
    sg_sym, sg_num
)

In [11]:
# Dope with partial occupancy 
structure_par = Structure.from_file("Mn0.2Zn0.8(FeO2)2.cif")
print(structure_par)
structure_par


Full Formula (Mn1.6 Zn6.4 Fe16 O32)
Reduced Formula: Mn1.6Zn6.4Fe16O32
abc   :   8.230723   8.230723   8.230723
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (56)
  #  SP                     a         b         c
---  --------------  --------  --------  --------
  0  Mn:0.2, Zn:0.8  0         0         0.5
  1  Mn:0.2, Zn:0.8  0         0.5       0
  2  Mn:0.2, Zn:0.8  0.5       0.5       0.5
  3  Mn:0.2, Zn:0.8  0.5       0         0
  4  Mn:0.2, Zn:0.8  0.75      0.25      0.25
  5  Mn:0.2, Zn:0.8  0.25      0.25      0.75
  6  Mn:0.2, Zn:0.8  0.25      0.75      0.25
  7  Mn:0.2, Zn:0.8  0.75      0.75      0.75
  8  Fe              0.125     0.125     0.125
  9  Fe              0.875     0.375     0.625
 10  Fe              0.375     0.625     0.875
 11  Fe              0.625     0.875     0.375
 12  Fe              0.375     0.875     0.625
 13  Fe              0.875     0.625     0.375
 14  Fe              0.625     0.375     0.875
 15  F

In [23]:
sga_loose = SpacegroupAnalyzer(structure_par, symprec=0.2, angle_tolerance=10)
sg_sym, sg_num = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()
print(
    sg_sym, sg_num
)

Fd-3m 227


In [25]:
calculator_1 = matcalc.load_fp("r2scan")

relax_calc = matcalc.RelaxCalc(
    calculator_1,
    optimizer="FIRE",
    relax_atoms=True,
    relax_cell=True
)

data = relax_calc.calc(structure_par)

ValueError: ASE Atoms only supports ordered structures

In [26]:
phonon_calc = PhononCalc(
    calculator_pbe,
    relax_structure=True,
    write_band_structure="output/ZFO_phonon_bs.yaml",
    write_total_dos="output/ZFO_phonon_dos.dat",
    write_phonon="output/ZFO_phonon.yaml",
    optimizer="FIRE",
)

data_ZFO = phonon_calc.calc(structure_par)

ValueError: ASE Atoms only supports ordered structures

## From MP

In [27]:
api_key = os.getenv("API_KEY")

mpr = MPRester(api_key)

# fetch the structure of Zn(FeO2)2
structure_mp = mpr.get_structure_by_material_id("mp-18750")
print(structure_mp)
structure_mp

Full Formula (Mn2 Fe4 O8)
Reduced Formula: Mn(FeO2)2
abc   :   5.822547   5.858924   5.818427
angles:  59.697325  59.781656  59.614076
pbc   :       True       True       True
Sites (14)
  #  SP            a          b         c    magmom
---  ----  ---------  ---------  --------  --------
  0  Mn     0.12509    0.12496   0.125294     4.321
  1  Mn     0.874911   0.87504   0.874705     4.321
  2  Fe     0.499999   0.500001  0.5          1.155
  3  Fe     0.499999   0.500001  0            1.175
  4  Fe    -0          0.500001  0.499999     1.173
  5  Fe     0.5       -0         0.5          1.163
  6  O      0.735222   0.73509   0.737108     0.015
  7  O      0.265265   0.26501   0.705124     0.002
  8  O      0.264676   0.707329  0.264014     0.009
  9  O      0.705603   0.263274  0.26782      0.001
 10  O      0.735325   0.292671  0.735985     0.009
 11  O      0.294397   0.736726  0.732179     0.001
 12  O      0.264776   0.26491   0.262892     0.015
 13  O      0.734733   0.73499   

In [35]:
sga_loose = SpacegroupAnalyzer(structure_mp, symprec=0.2, angle_tolerance=10)
sg_sym, sg_num = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()
print(
    sg_sym, sg_num
)

Fd-3m 227


In [32]:
from pymatgen.transformations.standard_transformations import SubstitutionTransformation

# Partial doping example: Replace Mn by 20%  + 80% Mn
substitution = {"Mn": {"Mn": 0.2, "Zn": 0.8}}

trans = SubstitutionTransformation(substitution)
doped_structure = trans.apply_transformation(structure_mp)

doped_structure

In [33]:
sga_loose = SpacegroupAnalyzer(doped_structure, symprec=0.2, angle_tolerance=10)
sg_sym, sg_num = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()
print(
    sg_sym, sg_num
)

Fd-3m 227


In [47]:
from pymatgen.transformations.standard_transformations import (
    DiscretizeOccupanciesTransformation,
    OrderDisorderedStructureTransformation,
)

# Step 1: Discretize small fractional occupancies
discretizer = DiscretizeOccupanciesTransformation(tol=0.1)
s_discretized = discretizer.apply_transformation(doped_structure)

# Optional: build supercell manually to increase multiplicities
# For example, building a 2x2x2 supercell to get integer occupancies
s_supercell = s_discretized * (2, 2, 2)  # multiply unit cell axes

# Step 2: Try ordering the discretized, possibly enlarged supercell
order_trans = OrderDisorderedStructureTransformation(no_oxi_states=True)
s_ordered = order_trans.apply_transformation(s_supercell, return_ranked_list=10)

for idx, ostruct in enumerate(s_ordered):
    print(f"-----Ordering {idx + 1}-----")
    print(ostruct)
    print("\n")

doped_structure = s_ordered[0]["structure"]
doped_structure

-----Ordering 1-----
{'energy': 0.0, 'energy_above_minimum': 0.0, 'structure': Structure Summary
Lattice
    abc : 11.645093912207724 11.717848591807579 11.636854749050878
 angles : 59.697325291675895 59.78165586172103 59.61407642092504
 volume : 1115.108107860193
      A : 10.07491636 0.05710324 5.83960716
      B : 3.397128 9.55859692 5.86521286
      C : 0.02443338 0.04890346 11.63672634
    pbc : True True True
PeriodicSite: Mn:0.2, Zn:0.8 (Mn) (10.95, 4.281, 18.95) [0.9375, 0.4375, 0.9374]
PeriodicSite: Mn:0.2, Zn:0.8 (Mn) (12.64, 9.036, 16.06) [0.9375, 0.9375, 0.4374]
PeriodicSite: Mn:0.2, Zn:0.8 (Mn) (12.65, 9.061, 21.88) [0.9375, 0.9375, 0.9374]
PeriodicSite: Mn:0.2, Zn:0.8 (Zn) (0.8439, 0.6039, 1.461) [0.06255, 0.06248, 0.06265]
PeriodicSite: Mn:0.2, Zn:0.8 (Zn) (0.8561, 0.6283, 7.279) [0.06255, 0.06248, 0.5626]
PeriodicSite: Mn:0.2, Zn:0.8 (Zn) (2.542, 5.383, 4.393) [0.06255, 0.5625, 0.06265]
PeriodicSite: Mn:0.2, Zn:0.8 (Zn) (2.555, 5.408, 10.21) [0.06255, 0.5625, 0.5626]
Pe

In [50]:
sga_loose = SpacegroupAnalyzer(doped_structure, symprec=0.2, angle_tolerance=10)
sg_sym, sg_num = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()
print(
    sg_sym, sg_num
)

R3m 160


In [ ]:
# Get back to R3m to fd3m

In [51]:
calculator_2 = matcalc.load_fp("r2scan")

relax_calc = matcalc.RelaxCalc(
    calculator_2,
    optimizer="FIRE",
    relax_atoms=True,
    relax_cell=True
)

data = relax_calc.calc(doped_structure)

In [52]:
final_structure_relaxed = data['final_structure']
print(final_structure_relaxed)
final_structure_relaxed

Full Formula (Mn3 Zn13 Fe32 O64)
Reduced Formula: Mn3Zn13(FeO2)32
abc   :  11.869711  11.933311  11.869822
angles:  59.785109  59.855524  59.696165
pbc   :       True       True       True
Sites (112)
  #  SP           a          b         c
---  ----  --------  ---------  --------
  0  Mn    0.936395   0.437564  0.937491
  1  Mn    0.937294   0.937953  0.436589
  2  Mn    0.936405   0.93827   0.93747
  3  Zn    0.060665   0.063326  0.062945
  4  Zn    0.062473   0.061701  0.563277
  5  Zn    0.06227    0.562725  0.062509
  6  Zn    0.06247    0.562794  0.563281
  7  Zn    0.56245    0.063716  0.063371
  8  Zn    0.562127   0.063593  0.560892
  9  Zn    0.562475   0.560578  0.063377
 10  Zn    0.560942   0.562762  0.563678
 11  Zn    0.438499   0.437653  0.43652
 12  Zn    0.43904    0.436838  0.93713
 13  Zn    0.438421   0.937157  0.436558
 14  Zn    0.437612   0.937807  0.936619
 15  Zn    0.937718   0.436664  0.438751
 16  Fe    0.249461   0.250689  0.25089
 17  Fe    0.250278   0.

In [53]:
sga_loose = SpacegroupAnalyzer(final_structure_relaxed, symprec=0.2, angle_tolerance=10)
sg_sym, sg_num = sga_loose.get_space_group_symbol(), sga_loose.get_space_group_number()
print(
    sg_sym, sg_num
)

R3m 160


In [ ]:
phonon_calc = PhononCalc(
    calculator_2,
    relax_structure=True,
    write_band_structure="output/MZFO_phonon_bs.yaml",
    write_total_dos="output/MZFO_phonon_dos.dat",
    write_phonon="output/MZFO_phonon.yaml",
    optimizer="FIRE",
)

data_ph = phonon_calc.calc(final_structure_relaxed)

In [ ]:
import matplotlib.pyplot as plt

phonon_bs = data_ph['phonon'].band_structure

n_axes = sum(1 for c in phonon_bs.path_connections if not c)

fig, axs = plt.subplots(1, n_axes, figsize=(16, 5))

if n_axes == 1:
    axs = [axs]

for ax in axs[1:]:
    ax.set_ylabel('')
    ax.tick_params(axis='y', labelleft=False)

phonon_bs.plot(ax=axs)
fig.suptitle("Phonon band structure", fontsize=16)
plt.subplots_adjust(wspace=0.07)
plt.show()


In [ ]:
phonon_dos = data_ph["phonon"].total_dos

fig, ax = plt.subplots(figsize=(8, 5))
phonon_dos.plot(ax=ax)
fig.suptitle("Phonon DOS", fontsize=16)
plt.show()

In [ ]:
plt.plot(
    data_ph['thermal_properties']['temperatures'],
    data_ph['thermal_properties']['heat_capacity']
)

plt.xlabel("Temperature (K)")
plt.ylabel('Heat Capacity (J/mol-K)')
plt.title("Heat Capacity vs Temperature")
plt.show()